In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from genotypes import PRIMITIVES
from genotypes import Genotype
from operations import *
import numpy as np
from utils import get_tensor_at

# Mixed Op

In [2]:
class MixedOp(Model):
    """Makes mixed operations object
    """

    def __init__(self, C, stride):
        """Makes ops array of all the arrays

        Args:
            C (int): the current state
        """
        super(MixedOp, self).__init__()
        self._ops = []
        for primitive in PRIMITIVES:
                op = OPS[primitive](C, stride)
                if 'pool' in primitive:
                    if('avg' in primitive):
                        op = AvgPool3x3(C, stride)
                    elif('max' in primitive):
                        op = MaxPool3x3(C, stride)
                self._ops.append(op)
        
    def call(self, x, weights):
        """Converts the discrete set of operation into conitnuous mixed operation

        Args:
            x (tensor): can be tensor or array, (e.g. image-array)
            weights (tensor): tensor or array of Softmax probability of alphas

        Returns:
            tensor: sum of product(weights, operation(x))
        """
        op_on_x = self._ops[0](x)
        mask = tf.eye(int(weights.shape[0]), dtype=tf.bool)
        s = tf.zeros(op_on_x.shape, dtype=op_on_x.dtype)
        for i in range(len(self._ops)):
            s += get_tensor_at(weights, mask, i) * self._ops[i](x)
        return s

In [3]:
mix_op = MixedOp(3, stride=1)

W0914 20:54:15.656581 139868646782784 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/operations.py:7: The name tf.layers.MaxPooling2D is deprecated. Please use tf.compat.v1.layers.MaxPooling2D instead.

W0914 20:54:15.658084 139868646782784 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/operations.py:23: The name tf.layers.BatchNormalization is deprecated. Please use tf.compat.v1.layers.BatchNormalization instead.

W0914 20:54:15.659063 139868646782784 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/operations.py:6: The name tf.layers.AveragePooling2D is deprecated. Please use tf.compat.v1.layers.AveragePooling2D instead.

W0914 20:54:15.663521 139868646782784 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/operations.py:130: The name tf.layers.Conv2D is deprecated. Please use tf.compat.v1.layers.Conv2D instead.



In [4]:
weights = np.random.rand(8).astype(np.float32)
features = np.random.rand(2, 16, 16, 3).astype(np.float32)

init = tf.global_variables_initializer()

out = mix_op(features, weights)

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    sess.run(init)
    res = sess.run(out)
print(res.shape)

W0914 20:54:16.495389 139868646782784 deprecation_wrapper.py:119] From /home/mythrex/anaconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/autograph/converters/directives.py:117: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0914 20:54:16.907027 139868646782784 deprecation.py:506] From /home/mythrex/anaconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0914 20:54:17.348575 139868646782784 deprecation.py:323] From /home/mythrex/anaconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py:193: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
U

(2, 16, 16, 3)


# Cell

In [5]:
class Cell(Model):

    def __init__(self, steps, multiplier, C_prev_prev, C_prev, C, reduction, reduction_prev, upsample_prev):
        super(Cell, self).__init__()
        self.reduction = reduction

        if reduction_prev:
            self.preprocess0 = FactorizedReduce(C_prev_prev, C)
        elif upsample_prev:
            self.preprocess0 = FactorizedUp(C_prev_prev, C)
        else:
            self.preprocess0 = ReLUConvBN(C_prev_prev, C, 1, 1, 'same')
        self.preprocess1 = ReLUConvBN(C_prev, C, 1, 1, 'same')
        self._steps = steps
        self._multiplier = multiplier

        self._ops = []
        for i in range(self._steps):
            for j in range(2+i):
                stride = 2 if reduction and j < 2 else 1
                op = MixedOp(C, stride)
                self._ops.append(op)

    def call(self, s0, s1, weights):
    
        s0 = self.preprocess0(s0)
        s1 = self.preprocess1(s1)

        states = [s0, s1]
        offset = 0
    
        for i in range(self._steps):
            s = sum(self._ops[offset+j](h, weights[offset+j]) for j, h in enumerate(states))
            offset += len(states)
            states.append(s)

        return tf.concat(states[-self._multiplier:], axis=-1)

## Cell Test

In [6]:
cell = Cell(steps=4,
            multiplier=4,
            C_prev_prev=3,
            C_prev=3,
            C=3,
            reduction=False,
            reduction_prev=False,
            upsample_prev=False)

In [7]:
weights = tf.random_uniform([14,8], 0, 1)
s0 = tf.random_uniform([2, 16, 16, 3], 0, 255)
s1 = tf.random_uniform([2, 16, 16, 3], 0, 255)

init = tf.global_variables_initializer()
cell_out = cell(s0, s1, weights)

In [8]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    sess.run(init)
    res = sess.run(cell_out)

In [9]:
print(res.shape)

(2, 16, 16, 12)


# Upsample Cell

In [10]:
class UpsampleCell(Model):

    def __init__(self, steps, multiplier, C_prev_prev, C_prev, C):
        super(UpsampleCell, self).__init__()
        self.preprocess0 = ReLUConvBN(C_prev_prev, C, 1, 1, 'same')
        self.preprocess1 = ReLUConvBN(C_prev, C, 1, 1, 'same')
        self._steps = steps
        self._multiplier = multiplier
        self.UpConv = layers.Conv2DTranspose(C*self._multiplier, 
                                        kernel_size=3,
                                        strides=2,
                                        padding='same')
        self.reduction = False

    def call(self, s0, s1, weights):
        s0 = self.preprocess0(s0)
        s1 = self.preprocess1(s1)

        s0 = self.UpConv(s0)
        s1 = self.UpConv(s1)

        return s0 + s1


## Upsample Cell Test

In [11]:
up_cell = UpsampleCell(4, 4, 3, 3, 3)

In [12]:
s0 = np.random.rand(2, 16, 16, 3).astype(np.float32)
s1 = np.random.rand(2, 16, 16, 3).astype(np.float32)

init = tf.global_variables_initializer()

up_cell_out = up_cell(s0, s1, [])
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    sess.run(init)
    res = sess.run(up_cell_out)
print(res.shape)

(2, 32, 32, 12)


# Network

In [13]:
class Network(Model):

    def __init__(self, C, net_layers, criterion, steps=4, multiplier=4, stem_multiplier=3):
        super(Network, self).__init__()
        self._C = C
        self._criterion = criterion
        self._steps = steps
        self._multiplier = multiplier
        self.net_layers = net_layers

        C_curr = C*stem_multiplier

        # stem operation
        self.stem_op = tf.keras.Sequential()
        self.stem_op.add(tf.keras.layers.Conv2D(
            C_curr, kernel_size=3, padding='same', use_bias=False))
        self.stem_op.add(tf.keras.layers.BatchNormalization())

        C_prev_prev, C_prev, C_curr = C_curr, C_curr, C
        self.cells = []
        self.skip_ops = []

        reduction_prev = False

        # For reduction
        for i in range(self.net_layers):
            if i % 2 == 1:
                C_curr *= 2
                reduction = True
            else:
                reduction = False
                self.skip_ops += [SkipConnection(C_curr)]
            cell = Cell(steps, multiplier, C_prev_prev, C_prev, C_curr,
                        reduction,
                        reduction_prev,
                        upsample_prev=False)
            reduction_prev = reduction
            self.cells += [cell]

            C_prev_prev, C_prev = C_prev, multiplier*C_curr

        for i in range(self.net_layers-1):
            if i % 2 == 0:
                C_curr = C_curr // 2

                cell = UpsampleCell(steps, multiplier,
                                    C_prev_prev, C_prev, C_curr)
            else:
                cell = Cell(steps, multiplier, C_prev_prev, C_prev, C_curr,
                            reduction=False,
                            reduction_prev=False,
                            upsample_prev=True)
            self.cells += [cell]
            C_prev_prev, C_prev = C_prev, multiplier*C_curr

        self.softmaxConv = tf.keras.Sequential()
        self.softmaxConv.add(tf.keras.layers.Conv2D(
            1, kernel_size=1, strides=1, padding='same'))
        self.softmaxConv.add(Softmax())

        self._initialize_alphas()

    def _initialize_alphas(self):
        k = sum(1 for i in range(self._steps) for n in range(2+i))
        num_ops = len(PRIMITIVES)

        self.alphas_normal = tf.Variable(
            1e-3*tf.random.uniform([k, num_ops]), name='alphas_normal')
        self.alphas_reduce = tf.Variable(
            1e-3*tf.random.uniform([k, num_ops]), name='alphas_reduce')
        self._arch_parameters = [
            self.alphas_normal,
            self.alphas_reduce,
        ]

    def arch_parameters(self):
        return self._arch_parameters

    def new(self):
        model_new = Network(self._C, self.net_layers, self._criterion)
        for x, y in zip(model_new.arch_parameters(), self.arch_parameters()):
            x.assign(y)
        return model_new

    def _loss(self, logits, target):
        return self._criterion(logits, tf.to_float(target))

    def call(self, inp):
        s0 = s1 = self.stem_op(inp)
        self.arr = []
        ids = []
        pos = -1

        middle = self.net_layers - 1
        for i, cell in enumerate(self.cells):
            if cell.reduction:
                weights = tf.nn.softmax(self.alphas_reduce, axis=-1)
            else:
                weights = tf.nn.softmax(self.alphas_normal, axis=-1)

            s0, s1 = s1, cell(s0, s1, weights)

            if (i < middle and i % 2 == 0):
                self.arr.append(s1)
                ids.append(i)

            if (i > middle and i % 2 == 1):
                C_curr = s1.shape[1]
                s1 = self.skip_ops[-pos-1](self.arr[pos], s1)
                pos -= 1

        return self.softmaxConv(s1)

    def genotype(self):

        def _parse(weights):
            gene = []
            n = 2
            start = 0
            for i in range(self._steps):
                end = start + n
                W = weights[start:end].copy()
                edges = sorted(range(i + 2), key=lambda x: -max(
                    W[x][k] for k in range(len(W[x])) if k != PRIMITIVES.index('none')))[:2]
                for j in edges:
                    k_best = None
                    for k in range(len(W[j])):
                        if k != PRIMITIVES.index('none'):
                            if k_best is None or W[j][k] > W[j][k_best]:
                                k_best = k
                    gene.append((PRIMITIVES[k_best], j))
                start = end
                n += 1
            return gene

            gene_normal = _parse(tf.nn.softmax(
                self.alphas_normal, axis=-1).numpy())
            gene_reduce = _parse(tf.nn.softmax(
                self.alphas_reduce, dim=-1).numpy())

            concat = range(2+self._steps-self._multiplier, self._steps+2)
            genotype = Genotype(
                normal=gene_normal, normal_concat=concat,
                reduce=gene_reduce, reduce_concat=concat
            )
            return genotype

    def get_model_thetas(self):
        specific_tensor = []
        specific_tensor_name = []
        for var in self.trainable_weights:
            if not 'alphas' in var.name:
                specific_tensor.append(var)
                specific_tensor_name.append(var.name)
        return specific_tensor

## Network Test

In [14]:
criterion = tf.losses.sigmoid_cross_entropy
network = Network(3, 3, criterion)

W0914 20:54:50.001156 139868646782784 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/operations.py:205: The name tf.layers.Conv2DTranspose is deprecated. Please use tf.compat.v1.layers.Conv2DTranspose instead.



In [15]:
image = tf.random_uniform((1, 16, 16, 3), 0, 255)
init = tf.global_variables_initializer()
net_out = network(image)

In [16]:
with tf.Session() as sess:
    sess.run(init)
    tf.initialize_all_variables().run()
    res = sess.run(net_out)

In [25]:
res.shape

(1, 16, 16, 1)

In [9]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Testing Here!

In [ ]:
import numpy as np

In [ ]:
s0 = np.random.rand(2, 16, 16, 6).astype(np.float32)
conv = tf.keras.Sequential()
conv.add(ReLUConvBN(6, 6, 1, 1, 'same'))
conv.add(FactorizedReduce(6,6))
conv(s0)

In [ ]:
var = tf.Variable([[0.9], [0.5]])
var2 = tf.Variable([[0.9], [0.5]])

In [ ]:
Zero(2).name